<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js010_mixLJMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [9]:
#@title js010_mixLJMD2D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js010_mixLJMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js010_mixLJMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.03 created, last updated on 2018.11.12
//    ver 0.0.1  2019.01.11 v1, last updated on 2021.04.29
//    ver 0.0.2  2021.10.28 v2, last updated on 2021.10.28
//    ver 0.0.3  2023.02.22 v3, last updated on 2023.08.11
//
// --------------------  molecular dynamics 2D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    potential: Lennard-Jones V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
//           force F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
//    for fast calculation
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      register near particles reg[][] (see registerNearParticles()), near means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const mixLJMD2D = (function(){ // ====================  mixLJMD2D Module  ====================

	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_nMax = 1000;						// array max

	let g_molecKind1 = 2;						// 2:Ar (kind: 0:He 1:Ne 2:Ar 3:Kr 4:Xe 5:Hg)
	let g_molecKind2 = 4;						// 4:Xe
	let g_nParticles = 200;						// number of particles
	let g_sysTime = 0.0;						// (s) system time
	let g_timeStep = 5.0*1.0e-15;				// (s) time step
	let g_xMax = 6.0E-9;						// (m) x-Box size
	let g_yMax = 6.0E-9;						// (m) y-Box size
	let g_kineticEnergy = 0.0;					// (J) total kinetic energy
	let g_potentialEnergy = 0.0;				// (J) total potential energy
	let g_rCutoff = 1.0e-9;						// (m) force cutoff length
	let g_hh = 1.0e-12;							// (m) forceTable r-division

	const g_xx = dim1( g_nMax );				// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );				// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );				// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );				// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );				// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );				// (N) y-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );				// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );			// kind of i-th particle 0:He 1:Ne 2:Ar 3:Kr 4:Xe 5:Hg
	const g_potentialTable = dim3( 6, 6, 1002 );// potential table [V[0], V[hh], V[2hh],..., V[rCutoff] ]
	const g_forceTable = dim3( 6, 6, 1002 );	// force table [F[0], F[hh], F[2hh],..., F[rCutoff] ]
	const g_reg = dimInt2( g_nMax, 100 );		// reg[i][0]:total number of particles near i-th particle
												// reg[i][j]:particle number near i-th particle, (j>0)

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  Lennard-Jones potential data  --------------------

	const g_LJMolec = [
	//    mass(kg)        E(J)         sigma(m)   string  color
		[   4.003*g_AMU,  10.2*g_kB, 2.576e-10, "He",  "#ee4444" ], // 0 He
		[  20.183*g_AMU,  36.2*g_kB, 2.976e-10, "Ne",  "#eeaa00" ], // 1 Ne
		[  39.948*g_AMU, 124.0*g_kB, 3.418e-10, "Ar",  "#aadd00" ], // 2 Ar
		[  83.500*g_AMU, 190.0*g_kB, 3.610e-10, "Kr",  "#44dd44" ], // 3 Kr
		[ 131.300*g_AMU, 229.0*g_kB, 4.055e-10, "Xe",  "#0088ee" ], // 4 Xe
		[ 200.590*g_AMU, 851.0*g_kB, 2.898e-10, "Hg",  "#8800ee" ], // 5 Hg
		[   2.016*g_AMU,  33.3*g_kB, 2.968e-10, "H2",  "#ff8800" ], // 6 H2
		[  28.013*g_AMU,  91.5*g_kB, 3.681e-10, "N2",  "#ff0000" ], // 7 N2
		[  31.999*g_AMU, 113.0*g_kB, 3.433e-10, "O2",  "#ff0088" ], // 8 O2
		[  18.015*g_AMU, 809.1*g_kB, 2.641e-10, "H2O", "#ff00ff" ], // 9 H2O
		[  16.043*g_AMU, 137.0*g_kB, 3.822e-10, "CH2", "#bb00ff" ], //10 CH4
		[  44.010*g_AMU, 190.0*g_kB, 3.996e-10, "CO2", "#6600ff" ], //11 CO2
		[  28.011*g_AMU, 110.0*g_kB, 3.590e-10, "CO",  "#0000ff" ]  //12 CO
	];

	const g_rCollision = [];
	const g_rBond = [];
	const g_massOf = [];
	const g_strOf = [];
	const g_colorOf = [];

	(function() {
		const n=g_LJMolec.length;

		for (let i=0; i<n; i++) {
			g_rCollision[i] = 0.5*g_LJMolec[i][2];
			g_rBond[i] = 1.12246*0.5*g_LJMolec[i][2];
			g_massOf[i] = g_LJMolec[i][0];
			g_strOf[i] = g_LJMolec[i][3];
			g_colorOf[i] = g_LJMolec[i][4];
		}
	}());

	//--- set forceTable[kind1][kind2][ir] and potentialTable[kind1][kind2][ir]

	function setForceTable() {
		const hh2=2.0*g_hh;

		for (let i=0; i<6; i++) {
			for (let j=0; j<6; j++) {
				const ep = Math.sqrt(g_LJMolec[i][1]*g_LJMolec[j][1]);
				const sg = 0.5*(g_LJMolec[i][2]+g_LJMolec[j][2]);
				for (let ir=1; ir<=1001; ir++) {
					const r = ir*g_hh;
					const ri = (sg/r);
					const r6 = ri*ri*ri*ri*ri*ri;
					g_potentialTable[i][j][ir] = cutoff(r)*4.0*ep*r6*(r6-1.0);
				}
				g_potentialTable[i][j][0] = g_potentialTable[i][j][1] + g_potentialTable[i][j][2];
				for (let ir=1; ir<=1000; ir++) {
					g_forceTable[i][j][ir] = -(g_potentialTable[i][j][ir+1] - g_potentialTable[i][j][ir-1])/hh2;
				}
				g_forceTable[i][j][1001] = -(0.0 - g_potentialTable[i][j][1000])/hh2;
				g_forceTable[i][j][0] = g_forceTable[i][j][1];
			}
		}
	}

	function cutoff(r) {
		let ret=0.0;

		if (r>=0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme, nn, BoxSizeInNM, contTemp ) {
		// theme  0:Ne 1:Ar 2:Kr 3:Xe 4:Hg 5:Ar-Xe 6:Ar-Hg 7:Ne-Kr 8:Ne-Ar
		let knd1,knd2,nn1,nn2;

		if (theme==0) { // Ne (knd: 1:Ne,2:Ar,3:Kr,4:Xe,5:Hg)
			knd1 = 1; nn1=nn; knd2 = 1; nn2 = 0;
		} else if (theme==1) { // Ar
			knd1 = 2; nn1=nn; knd2 = 2; nn2 = 0;
		} else if (theme==2) { // Kr
			knd1 = 3; nn1=nn; knd2 = 3; nn2 = 0;
		} else if (theme==3) { // Xe
			knd1 = 4; nn1=nn; knd2 = 4; nn2 = 0;
		} else if (theme==4) { // Hg
			knd1 = 5; nn1=nn; knd2 = 5; nn2 = 0;
		} else if (theme==5) { // Ar-Xe
			knd1 = 2; nn1 = nn/2; knd2 = 4; nn2 = nn/2;
		} else if (theme==6) { // Ar-Hg
			knd1 = 2; nn1 = nn/2; knd2 = 5; nn2 = nn/2;
		} else if (theme==7) { // Ne-Kr
			knd1 = 1; nn1 = nn/2; knd2 = 3; nn2 = nn/2;
		} else if (theme==8) { // Ne-Ar
			knd1 = 1; nn1 = nn/2; knd2 = 2; nn2 = nn/2;
		}

		g_sysTime = 0.0;
		g_molecKind1 = knd1
		g_molecKind2 = knd2
		setForceTable();
		g_xMax = BoxSizeInNM*1.0e-9;
		g_yMax = BoxSizeInNM*1.0e-9;
		g_nParticles = setGas(knd1,nn1,knd2,nn2);
		ajustVelocity(contTemp);
	}

	function setGas(knd1,nn1,knd2,nn2) {
		const nn=nn1+nn2;

		let nParticles = nn;
		for (let j=0; j<nn; j++) {
			g_kind[j] = (j<nn1) ? knd1 : knd2;
			const sgmj = 2.0*g_rCollision[g_kind[j]];
			let i;
			let loopCount = 0;
			do {
				g_xx[j] = (g_xMax-1.2*sgmj)*Math.random() + 0.6*sgmj;
				g_yy[j] = (g_yMax-1.2*sgmj)*Math.random() + 0.6*sgmj;
				for ( i=0; i<j; i++) {
					const d = Math.sqrt((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]));
					if (d < 1.2*(g_rCollision[g_kind[i]]+g_rCollision[g_kind[j]])) break;
				}
				loopCount = loopCount + 1
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nParticles = j - 1;
				break;
			}
		}
		for (let i=0; i<nParticles; i++) {
			setParticle(i, g_kind[i], g_xx[i], g_yy[i]);
		}
		return nParticles;
	}

	function setParticle(i, knd, x, y) {
		g_xx[i] = x;
		g_yy[i] = y;
		g_vx[i] = 200.0*normalRandom3();
		g_vy[i] = 200.0*normalRandom3();
		g_ffx[i] = 0.0;
		g_ffy[i] = 0.0;
		g_kind[i] = knd;
		g_mas[i] = g_massOf[g_kind[i]];
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc, tempMode, contTemp ) {
		for (let ii=0; ii<nCalc; ii++) {
			if (tempMode==1) ajustVelocity(contTemp);
			registerNearParticles();
			for (let i=0; i<20; i++) {
				g_sysTime += g_timeStep;
				moveParticles(g_timeStep);
			}
		}
	}

	function moveParticles(dt) {
		const nn=g_nParticles;

		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10; // Ar sigma=3.418e-10
		g_potentialEnergy = 0.0;
		for(let i=0; i<nn; i++) {
			g_ffx[i]=0;g_ffy[i]=0;
		}

		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j];
				const yij=g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij;
					const fyij = f*yij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function force(r,ki,kj) { // forceTable - linear interporation
		const ir = Math.floor(r/g_hh);
		const a = r - ir*g_hh;
		g_potentialEnergy += ((g_hh-a)*g_potentialTable[ki][kj][ir] + a*g_potentialTable[ki][kj][ir+1])/g_hh;
		return ((g_hh-a)*g_forceTable[ki][kj][ir] + a*g_forceTable[ki][kj][ir+1])/g_hh;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0);
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	//--- registeration

	function registerNearParticles() {
		const nn=g_nParticles;

		const rCut = g_rCutoff+20*2000*g_timeStep;
		const rcut2 = rCut*rCut;
		for (let i=0; i<nn-1; i++) {
			let k = 1;
			for (let j=i+1; j<nn; j++) {
				const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
				if (r2<rcut2) {
					g_reg[i][k] = j;
					k = k + 1;
				}
			}
			g_reg[i][0] = k;
		}
	}

	function maxNearParticles() {
		const nn=g_nParticles;
		let mx=0;
		for (let i=1; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek= 0.0;       //kinetic energy (J)

		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme, nn, BoxSizeInNM, contTemp )
		evolve:			timeEvolution,			// timeEvolution( ncalc, tempMode, contTemp)

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() { return [ g_molecKind1, g_molecKind2, g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getMolecStr:	function() { return [ g_strOf[g_molecKind1], g_strOf[g_molecKind2] ]; },
		getMolecData:	function(i) {
							const kind = g_kind[i];
							return [ kind, g_rCollision[kind] ];
						},
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})(); // ====================  mixLJMD2D end  ====================


const js010 = (function(){ // ====================  js Module  ====================

	const theModule = mixLJMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 8;			// theme  0:Ne 1:Ar 2:Kr 3:Xe 4:Hg 5:Ar-Xe 6:Ar-Hg 7:Ne-Kr 8:Ne-Ar
	let v_nn = 200;				// number of max. settable molecules
	let v_BoxSizeInNM = 12.0;
	let v_nCalc = 2;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control
	let v_contTemp = 300.0;

	let p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();
	let sysTime, temperature, kineticEnergy, potentialEnergy;

	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
  let breakFlag = false;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_nn, v_BoxSizeInNM, v_contTemp );
			[ p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
		}

		if ( !pauseFlag ) {
				theModule.evolve( v_nCalc, v_tempMode, v_contTemp );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( 1, v_tempMode, v_contTemp );
		}

		draw( ctx );

		requestAnimationFrame(animate);
	}


	// --------------------  draw  --------------------

	const molecColor = [ /*0:He*/ '#ff5050', /*1:Ne*/ '#ffb400', /*2:Ar*/ '#b4ff00',
						 /*3:Kr*/ '#50ff50', /*4:Xe*/ '#50b4ff', /*5:Hg*/ '#8000ff' ];

	function draw( ctx ) {
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();
		let molecStr1, molecStr2;
		[ molecStr1, molecStr2 ] = theModule.getMolecStr();

		let xp = 40, yp = 5, xSize = 400, ySize = 400, yTextPos= yCanvasSize - 45;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		drawBalls(ctx, p_nParticles, p_yMax, xp, yp, xSize/p_xMax );
		// caption
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (K)`, xp+200, yTextPos);
		let molecStr = molecStr1;
		if ( molecStr2 != molecStr ) molecStr += ", " + molecStr2;
		ctx.fillText(`molec:${molecStr}, N = ${p_nParticles}`, xp, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)}x${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, xp+200, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function drawBalls( ctx, nn, yMax, xp, yp, scale ) {
		const twoPi = 2.0*Math.PI;
		for (let i=0; i<nn; i++) {
			let x, y, kind, r;
			[ x, y ] = theModule.getPosition(i);
			[ kind, r ] = theModule.getMolecData(i);
			ctx.fillStyle = molecColor[kind];
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, r*scale, 0, twoPi, false);
			ctx.fill();
		}
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setnParticles() {
		const n = 1 + document.getElementById("slct_nn").selectedIndex;
		v_nn = n*100;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setNcalc() {
		v_nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature, kineticEnergy, potentialEnergy ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setnParticles:	setnParticles,	// setnParticles()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setNcalc:		setNcalc,		// setNcalc()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
	};

})(); // ====================  js010 module end  ====================


const js = js010;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js010]  mixed gas - Lennard-Jones molecular dynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Ne</option><option>Ar</option><option>Kr</option><option>Xe</option><option>Hg</option>
<option>Ar-Xe</option><option>Ar-Hg</option><option>Ne-Kr</option><option selected>Ne-Ar</option>
</select>
</select>
    <span style="margin-right: 50px;"></span>
<label>N=</label>
<select id="slct_nn" onChange="js.setnParticles()">
<option>100</option><option selected>200</option><option>300</option>
<option>400</option><option>500</option>
</select>
    <span style="margin-right: 50px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont. Temp</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<br>

<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option>1</option><option selected>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# python control
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(
    "i = {},\t".format(i),
    "time = {:.2f} (ps), ".format(sysTime*1e12),
    "temp = {:.2f} (K), ".format(sysTemp),
    "energy = {:.5g} (J)".format(energy) )
  time.sleep(2)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")


In [ ]:
# exec html-js code, and python control / change potential, dispMode

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)


# change theme and dispMode
themeList = [ '0:Ne', '1:Ar', '2:Kr', '3:Xe', '4:Hg', '5:Ar-Xe', '6:Ar-Hg', '7:Ne-Kr', '8:Ne-Ar',  ]
dispModeList = [ 'ball', 'bond length', 'bond direction' ]
for theme in [ 0, 3, 4, 5, 6, 7, 8 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  time.sleep(2)
  # get data and print
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(theme) )
  energy = kineticEnergy + potentialEnergy
  print(
    "-- time = {:.2f} (ps), ".format(sysTime*1e12),
    "temp = {:.2f} (K), ".format(sysTemp),
    "energy = {:.5g} (J)".format(energy) )
  time.sleep(1)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / system cool down and plot

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)


# change theme 1:Ar
eval_js( 'js.pysetTheme(1)' )
print( "-- theme:", themeList[1], "--" )


# python control

timeList = []
tempList = []
energyList = []
kkList = []
uuList = []

for i in range(20):
  # set temperature
  temp = 300 - 15*i
  eval_js( 'js.pysetTemperature({})'.format(temp) )
  time.sleep(3)

  # get data and display
  [ sysTime, sysTemp, kk, uu ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kk + uu

  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)
  kkList.append(kk*1e20)
  uuList.append(uu*1e20)

  if i%2==0:
    print(
      "i = {},\t".format(i),
      "time = {:.2f} (ps), ".format(sysTime*1e12),
      "temp = {:.2f} (K), ".format(sysTemp),
      "energy(K+U) = {:.5g} (J), ".format(energy),
      "K = {:.5g} (J), ".format(kk),
      "U = {:.5g} (J)".format(uu) )
  time.sleep(3)
#
time.sleep(3)


# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# plot
plt.plot( timeList, tempList, 'or-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

plt.plot( timeList, energyList, 'or-' )
plt.plot( timeList, kkList, '^g-' )
plt.plot( timeList, uuList, 'xb-' )
plt.xlabel('time (ps)')
plt.ylabel('energy (x 1e-20 J)')
plt.show()